# Import des fichiers de mesure des compteurs vélo via l'API Grand Lyon

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [11]:
import requests

X = 1000000
Y = 1
step = 1000000

while True:
    url = f"https://download.data.grandlyon.com/ws/timeseries/pvo_patrimoine_voirie.pvocomptagemeasure/all.csv?maxfeatures={X}&start={Y}"
    
    response = requests.get(url)
    
    if response.status_code == 200:
        # Save the response to a file
        filename = f"/content/drive/MyDrive/Colab Notebooks/mesures_compteurs_velos/response_{Y}_{X+Y-1}.csv"
        with open(filename, 'w') as file:
            file.write(response.text)
        
        # Increase X and Y by the step value
        Y += step
    else:
        break


Export to GCS

In [15]:
from google.colab import auth
auth.authenticate_user()
project_id = 'velyon-batch-1187'
!gcloud config set project {project_id}
!gsutil ls

Updated property [core/project].
gs://velyon_batch_1187/


Copy files using gsutil. Use -m tag for multi-threading to increase speed. (There is a Subfolder called "My Drive" that you have to address in your mounted drive)

In [ ]:
!gsutil -m cp -r /content/drive/MyDrive/Colab_Notebooks/mesures_compteurs_velos/* gs://velyon_batch_1187/Comptages/mesures/

Merge into a single CSV

In [31]:
import pandas as pd

folder_path = '/content/drive/MyDrive/Colab_Notebooks/mesures_compteurs_velos'

merged_df = pd.DataFrame()


In [ ]:
import os

for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv'):
        file_path = os.path.join(folder_path, file_name)
        df = pd.read_csv(file_path)
        merged_df = merged_df.append(df, ignore_index=True)


In [34]:
output_file_path = '/content/drive/MyDrive/Colab_Notebooks/mesures_compteurs_velos/merged_mesures.csv'
merged_df.to_csv(output_file_path, index=False)


In [36]:
#send to GCS

!gsutil -m cp -r /content/drive/MyDrive/Colab_Notebooks/mesures_compteurs_velos/merged_mesures.csv gs://velyon_batch_1187/Comptages/mesures/

Copying file:///content/drive/MyDrive/Colab_Notebooks/mesures_compteurs_velos/merged_mesures.csv [Content-Type=text/csv]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

Resuming upload for file:///content/drive/MyDrive/Colab_Notebooks/mesures_compteurs_velos/merged_mesures.csv
|
Operation completed over 1 objects/2.2 GiB.                                      
